In [1]:
#import pandas as pd
!wget http://www-public.tem-tsp.eu/~zhang_da/pub/dataset_tsmc2014.zip
!unzip dataset_tsmc2014.zip

--2024-05-31 09:04:18--  http://www-public.tem-tsp.eu/~zhang_da/pub/dataset_tsmc2014.zip
Resolving www-public.tem-tsp.eu (www-public.tem-tsp.eu)... 157.159.10.107, 2001:660:3203:100:1:0:80:107
Connecting to www-public.tem-tsp.eu (www-public.tem-tsp.eu)|157.159.10.107|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25546284 (24M) [application/zip]
Saving to: ‘dataset_tsmc2014.zip’

dataset_tsmc2014.zi 100%[===================>]  24.36M  5.65MB/s    in 4.3s    

2024-05-31 09:04:22 (5.65 MB/s) - ‘dataset_tsmc2014.zip’ saved [25546284/25546284]

Archive:  dataset_tsmc2014.zip
   creating: dataset_tsmc2014/
  inflating: dataset_tsmc2014/dataset_TSMC2014_NYC.txt  
  inflating: dataset_tsmc2014/dataset_TSMC2014_readme.txt  
  inflating: dataset_tsmc2014/dataset_TSMC2014_TKY.txt  


In [2]:
import pandas as pd
columns = ['User ID',
 'Venue ID',
 'Venue category ID',
 'Venue category name',
 'Latitude',
 'Longitude',
 'Timezone',
 'UTC time']

In [3]:
NUMCHECKIN = 7000

df = pd.read_csv('dataset_tsmc2014/dataset_TSMC2014_NYC.txt', sep='\t', encoding='latin-1', names=columns)
df = df.sample(NUMCHECKIN)
print(df)

        User ID                  Venue ID         Venue category ID  \
78066       371  4f7147c2e4b005ed80fa2d4f  4bf58dd8d48988d103941735   
66896       815  4a549b96f964a52027b31fe3  4bf58dd8d48988d198941735   
221865      837  4d717580117e3704768c9fb7  4bf58dd8d48988d10c951735   
36090        45  4a78d49ef964a52087e61fe3  4bf58dd8d48988d1f9941735   
180861      914  503e61efe4b02cb326f4a87e  4bf58dd8d48988d103941735   
...         ...                       ...                       ...   
214475      344  4bc5c26e5935c9b6984ca6d2  4bf58dd8d48988d1a1941735   
47782        40  4ca3b7a67f84224bf834c758  4bf58dd8d48988d16a941735   
206684      992  4b9e568cf964a52006db36e3  4bf58dd8d48988d1ff941735   
114582      316  4bc1bfa34cdfc9b6c2019521  4bf58dd8d48988d118951735   
42926       742  4ef7ce55c512f22778e7e0a4  4bf58dd8d48988d103941735   

              Venue category name   Latitude  Longitude  Timezone  \
78066              Home (private)  40.781685 -73.788969      -240   
66896   C

In [4]:

checkin_counts = df['User ID'].value_counts() # Conta il numero di check-in per ogni utente
users_with_sufficient_checkins = checkin_counts[checkin_counts >= 5].index   # Filtra gli utenti con meno di 5 check-in
df = df[df['User ID'].isin(users_with_sufficient_checkins)]  # Mantieni solo le righe relative agli utenti con almeno 5 check-in

print(f"Numero di utenti con meno di 5 check-in: {len(checkin_counts[checkin_counts < 5])}")

Numero di utenti con meno di 5 check-in: 452


In [5]:
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()
df['Venue ID'] = le.fit_transform(df['Venue ID'])
df['Venue category ID'] = le.fit_transform(df['Venue category ID'])
df.drop(['Venue category name'],axis=1,inplace=True)
print(df)

        User ID  Venue ID  Venue category ID   Latitude  Longitude  Timezone  \
78066       371      3689                  5  40.781685 -73.788969      -240   
66896       815       652                165  40.808612 -73.961549      -240   
221865      837      2806                 23  40.717862 -73.995017      -300   
36090        45       769                256  40.749916 -73.998696      -240   
180861      914      3894                  5  40.824530 -73.812498      -300   
...         ...       ...                ...        ...        ...       ...   
214475      344      1880                171  40.737116 -73.817731      -300   
47782        40      2539                129  40.760328 -73.826649      -240   
206684      992      1690                267  40.590192 -73.939730      -300   
114582      316      1862                 44  40.664685 -74.231765      -240   
42926       742      3504                  5  40.704774 -73.941860      -240   

                              UTC time 

In [6]:
import math

def distance_between_coordinates(lat1, lon1, lat2, lon2):
    radius = 6371  # Radius of the Earth in km

    # Convert latitude and longitude from degrees to radians
    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    # Calculate the differences in latitude and longitude<
    d_lat = lat2 - lat1
    d_lon = lon2 - lon1

    # Calculate the distance using the Haversine formula
    a = math.sin(d_lat/2) * math.sin(d_lat/2) + math.cos(lat1) * math.cos(lat2) * math.sin(d_lon/2) * math.sin(d_lon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = radius * c

    return distance

In [7]:
print(distance_between_coordinates(40.719810,-74.002581,40.735981, -74.029309))

2.8819941850086326


In [8]:
#build an alternative temporal graph
#two pois are connected if users go on average to them in the seme timeframe

df['localTime'] = pd.to_datetime(df['UTC time']) + pd.to_timedelta(df['Timezone'], unit = 'm')

df['localDayofWeek'] = df.localTime.dt.dayofweek
df['localHour'] = df.localTime.dt.hour






<ipython-input-8-f85843e15d2c>:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['localTime'] = pd.to_datetime(df['UTC time']) + pd.to_timedelta(df['Timezone'], unit = 'm')


In [9]:
venuesdf = df[['Venue ID','Latitude','Longitude','Venue category ID']].drop_duplicates(subset=['Venue ID'])
numvenues = len(venuesdf)
print(venuesdf)


        Venue ID   Latitude  Longitude  Venue category ID
78066       3689  40.781685 -73.788969                  5
66896        652  40.808612 -73.961549                165
221865      2806  40.717862 -73.995017                 23
36090        769  40.749916 -73.998696                256
180861      3894  40.824530 -73.812498                  5
...          ...        ...        ...                ...
141944      3829  40.851592 -73.930623                280
125888      1274  40.722349 -73.987707                 40
206684      1690  40.590192 -73.939730                267
114582      1862  40.664685 -74.231765                 44
42926       3504  40.704774 -73.941860                  5

[3967 rows x 4 columns]


In [10]:
!pip3 install torch --index-url https://download.pytorch.org/whl/${CUDA}
import torch
print(torch.__version__)
print(torch.version.cuda)
!pip install torch-geometric \
  torch-sparse \
  torch-scatter \
  torch-cluster \
  -f https://pytorch-geometric.com/whl/torch-2.3.0+cu121.html

Looking in indexes: https://download.pytorch.org/whl/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [11]:
import networkx as nx
venuesdict = venuesdf.to_dict('records')

i = 0
nodeIds = {}
node_attr = []

for el in venuesdict:
  #print(el['Venue ID'])
  nodeIds[el['Venue ID']] = i
  node_attr.append([float(el['Venue category ID'])])
  i+=1

print(len(node_attr))


3967


In [12]:
#import torch
sequences = {}
count = {}

MINVISIT = 5


for _, row in df.iterrows():

  if row['User ID'] not in sequences:
    sequences[row['User ID']] = [[row['Venue ID'],row['localHour'],row['localDayofWeek']]]


  else:
    sequences[row['User ID']].append([row['Venue ID'],row['localHour'],row['localDayofWeek']])


  if row['Venue ID'] not in count:
    count[row['Venue ID']] = 1
  else:
    count[row['Venue ID']] += 1

print(sequences)
seq = []
next = []
to_remove = []

for el in sequences:
  entry = [el]
  for i in range(len(sequences[el][len(sequences[el])-1])):
    el2 = sequences[el][len(sequences[el])-1][i]

    if i == 0:
      el2 = nodeIds[el2]
  
    entry.append(el2)
  for i in range(len(sequences[el])-1):
    entry.append(nodeIds[sequences[el][i][0]])
  seq.append(torch.tensor(entry))

  target = torch.zeros(numvenues)
  target[sequences[el][len(sequences[el])-1][0]] = 1
  next.append(target)
  if count[sequences[el][len(sequences[el])-1][0]] < MINVISIT:

    to_remove.append(sequences[el][len(sequences[el])-1][0])

seq=torch.nn.utils.rnn.pad_sequence(seq,batch_first=True)
seq_dim = len(seq[0])-4
print(seq_dim)
print(len(to_remove))
print(seq)
print(len(next))



{371: [[3689, 11, 0], [3593, 16, 0], [3555, 20, 1], [3746, 0, 1], [3806, 18, 0], [3746, 3, 6], [3289, 23, 0], [3288, 2, 0], [3746, 1, 0], [3689, 0, 5], [3583, 18, 4], [3289, 14, 4], [3442, 0, 3], [3815, 20, 6], [3289, 13, 0], [3689, 11, 6], [3289, 11, 6], [1246, 11, 1], [3746, 22, 5], [2388, 21, 0], [3289, 20, 2], [3065, 18, 5], [3288, 22, 2], [3593, 13, 5], [3442, 1, 0], [3746, 1, 0], [3078, 17, 5], [3078, 21, 4]], 815: [[652, 11, 0], [3513, 16, 5], [2610, 7, 0], [2734, 14, 6], [2576, 15, 6], [1773, 17, 2], [652, 8, 2], [2586, 14, 4], [1241, 8, 1], [3030, 19, 4]], 837: [[2806, 16, 6], [69, 20, 1], [3589, 18, 2], [2958, 22, 5], [306, 23, 1], [2638, 18, 5]], 45: [[769, 12, 2], [2846, 10, 3], [1102, 12, 2], [769, 10, 0], [2817, 10, 2], [2166, 17, 5], [503, 10, 2], [1827, 10, 3]], 914: [[3894, 19, 2], [3004, 9, 3], [3894, 19, 2], [3004, 8, 3], [3894, 20, 3], [2943, 7, 5]], 30: [[522, 2, 5], [2010, 2, 6], [1964, 15, 3], [2043, 17, 1], [2975, 18, 5], [457, 14, 6], [1023, 21, 2], [377, 17, 6

In [13]:
df = df[~df['Venue ID'].isin(to_remove)]
venuesdf = venuesdf[~venuesdf['Venue ID'].isin(to_remove)]
venuesdict = venuesdf.to_dict('records')
print(df)
print(venuesdf)

        User ID  Venue ID  Venue category ID   Latitude  Longitude  Timezone  \
78066       371      3689                  5  40.781685 -73.788969      -240   
66896       815       652                165  40.808612 -73.961549      -240   
221865      837      2806                 23  40.717862 -73.995017      -300   
36090        45       769                256  40.749916 -73.998696      -240   
180861      914      3894                  5  40.824530 -73.812498      -300   
...         ...       ...                ...        ...        ...       ...   
188956      315      2447                195  40.670920 -73.875493      -300   
17563       798      1618                 28  40.874973 -73.909020      -240   
108124      382      3052                299  40.812099 -73.909577      -240   
43794       742      1518                262  40.707561 -73.940256      -240   
32342       745      2167                 64  40.671416 -73.992871      -240   

                              UTC time 

In [14]:
#fast spatial graph construction
import networkx as nx
from tqdm import tqdm
import torch
from torch_geometric.data import Data

venuesdict = venuesdf.to_dict('records')
edge_index = []
edges_attr = []

for i in tqdm(range(len(venuesdict) - 1)):

   for j in range(i + 1, len(venuesdict)):

      if distance_between_coordinates(venuesdict[i]['Latitude'], venuesdict[i]['Longitude'],
                                    venuesdict[j]['Latitude'], venuesdict[j]['Longitude']) < 1:
        edge_index.append([nodeIds[venuesdict[i]['Venue ID']], nodeIds[venuesdict[j]['Venue ID']]])
        edges_attr.append([1])

spatG = Data(x=torch.tensor(node_attr), edge_index=torch.LongTensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(spatG)


100%|██████████| 3397/3397 [00:25<00:00, 132.14it/s] 


Data(x=[3967, 1], edge_index=[2, 136986], edge_attr=[136986, 1])


In [15]:
import pickle

print(spatG)
pickle.dump(spatG,open('spatG.pickle','wb'))

Data(x=[3967, 1], edge_index=[2, 136986], edge_attr=[136986, 1])


In [16]:
#nx.draw(G)
print(spatG)

Data(x=[3967, 1], edge_index=[2, 136986], edge_attr=[136986, 1])


In [17]:

#sequantial relation graph
#build a graph that captures the sequence of actions made by the users

import networkx as nx

edge_index = []
edges_attr = []

usrloc = {}


for _, row in df.iterrows():

  if row['User ID'] not in usrloc:
    usrloc[row['User ID']] = row['Venue ID']

  else:
    edge_index.append([nodeIds[usrloc[row['User ID']]], nodeIds[row['Venue ID']]])
    edges_attr.append(1)
    usrloc[row['User ID']] = row['Venue ID']


seqG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(seqG)

Data(x=[3967, 1], edge_index=[2, 4229], edge_attr=[4229])


In [18]:
import pickle


print(seqG)

pickle.dump(seqG,open('tempG.pickle','wb'))

#data = pickle.load(open('tempG.pickle','rb'))

Data(x=[3967, 1], edge_index=[2, 4229], edge_attr=[4229])


In [19]:
import math
groupeddf = df.groupby(['Venue ID','Venue category ID'],as_index=False)
avghour = groupeddf['localHour'].mean()
avgday = groupeddf['localDayofWeek'].mean()

print(avghour)

timedf = pd.concat([avghour,avgday['localDayofWeek']],axis=1)



print(timedf)

hours = {}
week = {}

for _, row in timedf.iterrows():

  hour = math.floor(row['localHour'])
  day = math.floor(row['localDayofWeek'])

  if hour not in hours:
    hours[hour] = [[row['Venue ID'],row['Venue category ID']]]

  else:
    hours[hour].append([row['Venue ID'],row['Venue category ID']])

  if day not in week:
    week[day] = [[row['Venue ID'],row['Venue category ID']]]

  else:
    week[day].append([row['Venue ID'],row['Venue category ID']])






      Venue ID  Venue category ID  localHour
0            0                231       19.0
1            1                 40        1.0
2            2                 40        0.0
3            3                 40       21.0
4            4                 22       22.0
...        ...                ...        ...
3411      3961                122       17.0
3412      3962                132       13.0
3413      3963                 85       13.0
3414      3965                298        0.0
3415      3966                267       16.0

[3416 rows x 3 columns]
      Venue ID  Venue category ID  localHour  localDayofWeek
0            0                231       19.0             4.0
1            1                 40        1.0             6.0
2            2                 40        0.0             6.0
3            3                 40       21.0             1.0
4            4                 22       22.0             0.0
...        ...                ...        ...             ...
3411    

In [20]:
#dayG = nx.Graph()
#hourG = nx.Graph()

edge_index = []
edges_attr = []

for el in hours:
   for u in hours[el]:

    for v in hours[el]:

      if u != v:
        edge_index.append([nodeIds[u[0]], nodeIds[v[0]]])

hourG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(hourG)




Data(x=[3967, 1], edge_index=[2, 609296], edge_attr=[0])


In [21]:
pickle.dump(hourG,open('hourG.pickle','wb'))

#data = pickle.load(open('tempG.pickle','rb'))

In [22]:
edge_index = []
edges_attr = []


for el in week:
   for u in week[el]:

    for v in week[el]:

      if u != v:
        edge_index.append([nodeIds[u[0]], nodeIds[v[0]]])

dayG = Data(x=torch.tensor(node_attr), edge_index=torch.tensor(edge_index).t().contiguous(), edge_attr=torch.tensor(edges_attr))
print(dayG)

Data(x=[3967, 1], edge_index=[2, 1688650], edge_attr=[0])


In [23]:
pickle.dump(dayG,open('dayG.pickle','wb'))

In [24]:
!pip install pytorch-lightning
import pytorch_lightning as pl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.3/802.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 17.9 MB/s eta 0:00:00


In [25]:
import matplotlib.pyplot as plt
import torch
from sklearn.manifold import TSNE
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

from torch_geometric.datasets import Planetoid
from torch_geometric.nn import Node2Vec

# Lightning Module for Node2Vec model
class Node2VecModel(pl.LightningModule):
    def __init__(self, data, embedding_dim=32, walk_length=20, context_size=10, walks_per_node=10, num_negative_samples=1, p=1.0, q=1.0, lr=0.01):
        super().__init__()
        self.data = data
        self.model = Node2Vec(
            data.edge_index,
            embedding_dim=embedding_dim,
            walk_length=walk_length,
            context_size=context_size,
            walks_per_node=walks_per_node,
            num_negative_samples=num_negative_samples,
            p=p,
            q=q,
            sparse=True
        )
        self.lr = lr

    def forward(self):
        return self.model()

    def training_step(self, batch, batch_idx):
        pos_rw, neg_rw = batch
        loss = self.model.loss(pos_rw.to(self.device), neg_rw.to(self.device))
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.SparseAdam(self.parameters(), lr=self.lr)

    def test_step(self, batch, batch_idx):
        z = self.model()
        acc = self.model.test(
            train_z=z[self.data.train_mask],
            train_y=self.data.y[self.data.train_mask],
            test_z=z[self.data.test_mask],
            test_y=self.data.y[self.data.test_mask],
            max_iter=150
        )
        self.log('test_acc', acc)
        return acc




batch_size = 128
num_workers = 4
graph_embed = []

for el in [spatG,hourG,seqG,dayG]:

  model = Node2VecModel(data=el)

  loader = model.model.loader(batch_size=128, shuffle=True, num_workers=num_workers)

  trainer = pl.Trainer(
      max_epochs=100,
      callbacks=[
          ModelCheckpoint(monitor='train_loss'),
          EarlyStopping(monitor='train_loss', patience=10)
      ]
  )
  trainer.fit(model, loader)
  #trainer.test(model, loader)

  # Plotting points


  model.eval()
  z = model().detach()
  graph_embed.append(z)

  #z = TSNE(n_components=2).fit_transform(z)
  print(z.shape)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type     | Params
-----------------------------------
0 | 

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type     | Params
-----------------------------------
0 | model | Node2Vec | 126 K 
-----------------------------------
126 K     Trainable params
0         Non-trainable params
126 K     Total params
0.505     Total estimated model params size (MB)


torch.Size([3947, 32])


Training: |          | 0/? [00:00<?, ?it/s]

...

Training: |          | 0/? [00:00<?, ?it/s]

torch.Size([3947, 32])


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type     | Params
-----------------------------------
0 | model | Node2Vec | 126 K 
-----------------------------------
126 K     Trainable params
0         Non-trainable params
126 K     Total params
0.505     Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

torch.Size([3947, 32])


In [26]:
from sklearn.model_selection import train_test_split

batchsize = 32

train_x,test_x,train_y,test_y = train_test_split(seq,torch.stack(next),test_size=0.2)
train_x,val_x,train_y,val_y = train_test_split(train_x,train_y,test_size=0.2)

trainset = torch.utils.data.TensorDataset(train_x,train_y)
trainloader = torch.utils.data.DataLoader(trainset,batch_size=batchsize,shuffle=True)
print(len(trainloader))

valset = torch.utils.data.TensorDataset(val_x,val_y)
valloader = torch.utils.data.DataLoader(valset,batch_size=batchsize,shuffle=True)
print(len(valloader))

testset = torch.utils.data.TensorDataset(test_x,test_y)
testloader = torch.utils.data.DataLoader(testset,batch_size=batchsize,shuffle=True)
print(len(testloader))

datasets = [trainset,valset,testset]

pickle.dump(datasets,open('datasets.pickle','wb'))


13
4
4


In [38]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import pytorch_lightning as pl
!pip install matplotlib

class GCN(pl.LightningModule):
    def __init__(self, in_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, 64)
        self.conv2 = GCNConv(64, out_channels)
        self.dropout = torch.nn.Dropout(p=0.5)  # Add dropout with 50% probability

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        return x

class LightningModel(pl.LightningModule):
    def __init__(self, input_dim, location_dim, sequence_dim, graph_embed):
        super(LightningModel, self).__init__()
        self.graph_embed = graph_embed
        self.train_losses = []
        self.test_losses = []
        self.val_losses = []
        self.train_accs = []
        self.val_accs = []
        self.test_accs = []
        self.train_acc_at_k = []
        self.val_acc_at_k = []
        self.test_acc_at_k =[]
        self.train_mrr = []
        self.val_mrr = []
        self.test_mrr = []

        # User Embedding Layer
        self.user_embedding = torch.nn.Embedding(num_embeddings=input_dim, embedding_dim=32)

        # Last location Embedding Layer
        self.lastLoc_embedding = torch.nn.Embedding(num_embeddings=input_dim, embedding_dim=32)

        # Time and day Input linear
        self.timeday_embedding = torch.nn.Linear(2, 32)

        # Sequence Embedding Layer
        self.self_attn = torch.nn.MultiheadAttention(embed_dim=sequence_dim, num_heads=1)

        # Expand sequence representation
        self.expand_sequence = torch.nn.Linear(sequence_dim, 32)

        # Multihead Attention Layer
        self.multihead_attn = torch.nn.MultiheadAttention(embed_dim=32, num_heads=2)

        # Classifier Layers
        self.linear1 = torch.nn.Linear(int((4*32*(self.graph_embed[0].shape[0]+batchsize))/batchsize), 128)
        self.dropout1 = torch.nn.Dropout(p=0.5)  # Add dropout with 50% probability
        self.linear2 = torch.nn.Linear(128, 64)
        self.dropout2 = torch.nn.Dropout(p=0.5)  # Add dropout with 50% probability
        self.linear3 = torch.nn.Linear(64, numvenues)

    def forward(self, user, lastloc, lasthour, lastday, sequence):
        user_embed = self.user_embedding(user.int())
        lastloc_embed = self.lastLoc_embedding(lastloc.int())
        timeday_embed = self.timeday_embedding(torch.stack([lasthour.float(), lastday.float()], dim=1))
        graph_embed = torch.cat(self.graph_embed)

        # Sequence Embedding
        sequence_embed, _ = self.self_attn(sequence.float(), sequence.float(), sequence.float())
        sequence_embed = self.expand_sequence(sequence_embed)

        # Concatenate all embeddings
        concat_embed = torch.cat([user_embed, lastloc_embed, graph_embed.to(self.device), timeday_embed, sequence_embed], dim=0)

        # Multihead Attention
        attn_output, _ = self.multihead_attn(concat_embed, concat_embed, concat_embed)

        # Classifier
        out = F.relu(self.linear1(attn_output.view(batchsize, -1)))
        out = self.dropout1(out)
        out = F.relu(self.linear2(out))
        out = self.dropout2(out)
        out = self.linear3(out)

        return out

    def training_step(self, batch, batch_idx):
        x, y = batch
        pad_dim = batchsize - x.shape[0]
        if pad_dim != 0:
            x = torch.cat([x, torch.zeros(pad_dim, x.shape[1]).to(self.device)])
            y = torch.cat([y, torch.zeros(pad_dim, y.shape[1]).to(self.device)])
        y_pred = self(x[:, 0], x[:, 1], x[:, 2], x[:, 3], x[:, 4:])
        loss = F.cross_entropy(y_pred, y)
        acc = self.calculate_accuracy(y_pred, y)
        acc_at_k_value = accuracy_at_k(y_pred, y)
        mrr_value = mean_reciprocal_rank(y_pred, y)
        self.log('train_loss', loss, on_step=False, on_epoch=True)
        self.log('train_acc', acc, on_step=False, on_epoch=True)
        self.log('train_acc_at_k', acc_at_k_value, on_step=False, on_epoch=True)
        self.log('train_mrr', mrr_value, on_step=False, on_epoch=True)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        pad_dim = batchsize - x.shape[0]
        if pad_dim != 0:
            x = torch.cat([x, torch.zeros(pad_dim, x.shape[1]).to(self.device)])
            y = torch.cat([y, torch.zeros(pad_dim, y.shape[1]).to(self.device)])
        y_pred = self(x[:, 0], x[:, 1], x[:, 2], x[:, 3], x[:, 4:])
        val_loss = F.cross_entropy(y_pred, y)
        acc_at_k = accuracy_at_k(y_pred, y, k=5)
        acc = self.calculate_accuracy(y_pred, y)
        mrr = mean_reciprocal_rank(y_pred, y)
        self.log('val_acc', acc, on_step=False, on_epoch=True)
        self.log('val_loss', val_loss, on_step=False, on_epoch=True)
        self.log('val_acc_at_k', acc_at_k, on_step=False, on_epoch=True)
        self.log('val_mrr', mrr, on_step=False, on_epoch=True)

        return val_loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        pad_dim = batchsize - x.shape[0]
        if pad_dim != 0:
            x = torch.cat([x, torch.zeros(pad_dim, x.shape[1]).to(self.device)])
            y = torch.cat([y, torch.zeros(pad_dim, y.shape[1]).to(self.device)])
        y_pred = self(x[:, 0], x[:, 1], x[:, 2], x[:, 3], x[:, 4:])
        test_loss = F.cross_entropy(y_pred, y)
        acc = self.calculate_accuracy(y_pred, y)
        acc_at_k = accuracy_at_k(y_pred, y, k=5)
        mrr = mean_reciprocal_rank(y_pred, y)
        self.log('test_loss', test_loss)
        self.log('test_acc', acc)
        self.log('test_acc_at_k', acc_at_k)
        self.log('test_mrr', mrr)

        return {'test_loss': test_loss, 'test_acc': acc, 'test_acc_at_k': acc_at_k, 'test_mrr': mrr}



    def on_train_epoch_end(self):
        # Collect training metrics once per epoch
        epoch_loss = self.trainer.callback_metrics.get('train_loss')
        epoch_acc = self.trainer.callback_metrics.get('train_acc')
        epoch_acc_at_k = self.trainer.callback_metrics.get('train_acc_at_k')
        epoch_mrr = self.trainer.callback_metrics.get('train_mrr')
        if epoch_loss is not None and epoch_acc is not None and epoch_acc_at_k is not None and epoch_mrr is not None:
            self.train_losses.append(epoch_loss.item())
            self.train_accs.append(epoch_acc.item())
            self.train_acc_at_k.append(epoch_acc_at_k.item())
            self.train_mrr.append(epoch_mrr.item())
        
        print(f"End of epoch {self.current_epoch} - Training loss: {epoch_loss.item()}, Accuracy@k: {epoch_acc_at_k.item()}, MRR: {epoch_mrr.item()}")

    def on_validation_epoch_end(self):
        # Collect validation metrics once per epoch
        epoch_val_loss = self.trainer.callback_metrics.get('val_loss')
        epoch_val_acc = self.trainer.callback_metrics.get('val_acc')
        epoch_val_acc_at_k = self.trainer.callback_metrics.get('val_acc_at_k')
        epoch_val_mrr = self.trainer.callback_metrics.get('val_mrr')
        if epoch_val_loss is not None and epoch_val_acc is not None and epoch_val_acc_at_k is not None and epoch_val_mrr is not None:
            self.val_losses.append(epoch_val_loss.item())
            self.val_accs.append(epoch_val_acc.item())
            self.val_acc_at_k.append(epoch_val_acc_at_k.item())
            self.val_mrr.append(epoch_val_mrr.item())
        
        print(f"End of epoch {self.current_epoch} - Validation loss: {epoch_val_loss.item()}, Accuracy@k: {epoch_val_acc_at_k.item()}, MRR: {epoch_val_mrr.item()}")


    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

    def train_dataloader(self):
        return trainloader

    def val_dataloader(self):
        return valloader

    def test_dataloader(self):
        return testloader

    def calculate_accuracy(self, y_pred, y_true):
        _, predicted = torch.max(y_pred, 1)
        y_true = y_true.argmax(dim=1)  # Convert one-hot to class indices if necessary
        correct = (predicted == y_true).float().sum()
        accuracy = correct / y_true.size(0)
        return accuracy

def accuracy_at_k(y_pred, y_true, k=5):
    _, top_k = y_pred.topk(k, dim=1)
    _, label = y_true.topk(1, dim=1)

    correct = top_k.eq(label.view(-1, 1).expand_as(top_k))
    acc_at_k = correct.float().sum(dim=1).mean()
    return acc_at_k

def mean_reciprocal_rank(y_pred, y_true):
    _, rank = y_pred.sort(dim=1, descending=True)
    rank = rank.argsort(dim=1)
    _, label = y_true.topk(1, dim=1)
    rr = (1.0 / (rank.gather(1, label.view(-1, 1).long()) + 1)).mean()
    return rr

input_dim = 10000
location_dim = 32
sequence_dim = seq_dim
model = LightningModel(input_dim, location_dim, sequence_dim,graph_embed)


trainer = pl.Trainer(
    max_epochs=50,
    log_every_n_steps=1,
    callbacks=[
        ModelCheckpoint(monitor='train_loss'),
        EarlyStopping(monitor='train_loss', patience=10)
    ]
)


trainer.fit(model, trainloader, valloader)

def plot_metrics(model):
    num_epochs = min(len(model.train_losses), len(model.val_losses))
    epochs = range(num_epochs)

    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, model.train_losses[:num_epochs], label='Training Loss')
    plt.plot(epochs, model.val_losses[:num_epochs], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Loss')

    plt.subplot(1, 2, 2)
    plt.plot(epochs, model.train_acc_at_k[:num_epochs], label='Training Accuracy')
    plt.plot(epochs, model.val_acc_at_k[:num_epochs], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Accuracy')

    plt.tight_layout()
    plt.show()


plot_metrics(model)



INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name              | Type               | Params
---------------------------------------------------------
0 | user_embedding    | Embedding          | 320 K 
1 | lastLoc_embedding | Embedding          | 320 K 
2 | timeday_embedding | Linear             | 96    
3 | self_attn         | MultiheadAttention | 29.9 K
4 | expand_sequence   | Linear             | 2.8 K 
5 | multihead_attn    | MultiheadAttention | 4.2 K 
6 | linear1           | Linear             | 2.0 M 
7 | linear2           | Linear             | 8.3 K 
8 | linear3        

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


tensor([[2218, 3787, 3337, 3338, 1030],
        [3781, 2001, 2077, 3056, 3684],
        [3832, 2447, 2218, 3216,  343],
        [1227, 3371, 1805,  362, 2300],
        [3182, 2265,  903, 2853, 3781],
        [2129, 2690, 3337, 2857, 3282],
        [ 571, 1801, 1279, 1385, 3412],
        [3313, 2315, 2605, 3781,  535],
        [1805, 1871,  163, 2046, 3518],
        [ 903, 3890, 3505,   24, 2005],
        [ 903, 3853, 3781, 3476, 1579],
        [1805,  550,  546, 1955,   79],
        [  24, 3182, 2207,  196, 2282],
        [1801, 3234, 3337, 3832, 2129],
        [1279, 1227, 3182,  362, 2723],
        [3412, 1613,  341, 1812, 2005],
        [1801,  549, 3337, 3732,  163],
        [3329, 3371, 2218, 3476, 3492],
        [3890, 2375, 2129, 1000, 2507],
        [ 921,  903,  362, 3691, 1126],
        [2562, 2368, 3846, 2005, 1389],
        [2001, 3239, 1178, 2324,  356],
        [2033, 2005, 3371, 1613, 2375],
        [2954, 1613, 2176, 2434, 2452],
        [3732, 3337, 3234, 2983, 2033],


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

tensor([[1227, 2571, 2385, 3890, 3246],
        [1227,  903, 2129, 3412, 3853],
        [1227, 3787, 2129, 3890, 2385],
        [3890, 1227, 3412, 1559,  903],
        [1227, 3787, 2385, 2005, 2129],
        [1227,   47, 2189, 3890, 3246],
        [1227, 2385, 2005, 2723, 3412],
        [1227,   47, 2129, 3246, 2189],
        [1227, 2571, 3890, 2385, 2129],
        [1227,  903, 2129, 3412, 3890],
        [1227, 3787, 2129, 3890, 2385],
        [3890, 3412, 1227, 1559,  903],
        [1227, 3787, 2005, 2385, 2129],
        [1227,   47, 2189, 3246, 3890],
        [1227, 2385, 2005, 2723, 3412],
        [1227,   47, 2129, 3246, 2189],
        [1227, 2571, 2385, 3890, 3246],
        [1227,  903, 2129, 3890, 3412],
        [1227, 3787, 2129, 3890, 2385],
        [1227,  903, 3412, 3890, 1559],
        [1227, 2005, 2385, 2129, 3787],
        [1227, 2189,   47, 3890, 3246],
        [1227, 2385, 2005, 2723, 3412],
        [1227,   47, 2129, 3246, 2189],
        [1227, 2571, 2385, 3890, 3246],


Validation: |          | 0/? [00:00<?, ?it/s]

tensor([[1227, 2571, 2385, 3890, 3246],
        [1227,  903, 3412, 2129, 3853],
        [1227, 3787, 2129, 3890, 2385],
        [1227, 3412,  903, 3890, 1559],
        [1227, 3787, 2005, 2385, 2129],
        [1227,   47, 2189, 3890, 3412],
        [1227, 2385, 2005, 3412, 1279],
        [1227,   47, 2189, 2129,  903],
        [1227, 2571, 3890, 2385, 3246],
        [1227,  903, 3412, 2129,   47],
        [1227, 3787, 2129, 3890, 3246],
        [1227, 3412,  903, 3890, 1559],
        [1227, 3787, 2005, 2385, 2129],
        [1227, 2189,   47, 3890, 3412],
        [1227, 2385, 2005, 3412, 1279],
        [1227,   47, 2129, 2189,  903],
        [1227, 2385, 2571, 3890, 3246],
        [1227,  903, 3412, 2129, 3853],
        [1227, 3787, 2129, 3890, 2385],
        [1227, 3412,  903, 3890, 1559],
        [1227, 2005, 3787, 2385, 2129],
        [1227,   47, 2189, 3890, 3412],
        [1227, 2385, 2005, 3412, 2723],
        [1227,   47, 2129, 3246, 2189],
        [1227, 2571, 2385, 3890, 3246],


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [ ]:
def plot_metrics(model):
    num_epochs = min(len(model.train_losses), len(model.val_losses))
    epochs = range(num_epochs)



    plt.figure(figsize=(12, 8))

    plt.subplot(2, 2, 1)
    plt.plot(epochs, model.train_losses[:num_epochs], label='Training Loss')
    plt.plot(epochs, model.val_losses[:num_epochs], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Loss')

    plt.subplot(2, 2, 2)
    plt.plot(epochs, model.train_accs[:num_epochs], label='Training Accuracy')
    plt.plot(epochs, model.val_accs[:num_epochs], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Accuracy')

    plt.subplot(2, 2, 3)
    plt.plot(epochs, model.train_acc_at_k[:num_epochs], label='Training Accuracy@k')
    plt.plot(epochs, model.val_acc_at_k[:num_epochs], label='Validation Accuracy@k')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy@k')
    plt.legend()
    plt.title('Accuracy@k')

    plt.subplot(2, 2, 4)
    plt.plot(epochs, model.train_mrr[:num_epochs], label='Training MRR')
    plt.plot(epochs, model.val_mrr[:num_epochs], label='Validation MRR')
    plt.xlabel('Epochs')
    plt.ylabel('MRR')
    plt.legend()
    plt.title('MRR')

    plt.tight_layout()
    plt.show()

    # Plot the metrics after training
plot_metrics(model)


In [ ]:
# Stampa i risultati del test

test_result = trainer.test(model, dataloaders=testloader)[0]

test_losses = model.test_losses
train_losses = model.train_losses
val_losses = model.val_losses
train_accs = model.train_accs
val_accs = model.val_accs
print(f"Test Loss: {test_result['test_loss']}")
print(f"Test Accuracy: {test_result['test_acc']}")
print(f"Test Accuracy@5: {test_result['test_acc_at_k']}")
print(f"Test MRR: {test_result['test_mrr']}")
